In [ ]:
%run ../../_pre_run.ipynb

# Data Exploration


## Table df_orders

Let’s look at the information about the dataframe.

In [ ]:
df_orders.explore.info()

### Initial Column Analysis

We will examine each column individually.

**order_id**

In [ ]:
df_orders.order_id.explore.info(plot=False)

**customer_id**

In [ ]:
df_orders.customer_id.explore.info(plot=False)

**order_status**

In [ ]:
df_orders.order_status.explore.info()

**Key Observations:**  

- 97% of all orders were delivered

**order_purchase_dt**

In [ ]:
df_orders.order_purchase_dt.explore.info()

**Key Observations:**  

- In order_purchase_dt missing 4% of months, 10% of weeks, 18% of days


**order_approved_dt**

In [ ]:
df_orders.order_approved_dt.explore.info()

**Key Observations:**  

- In order_approved_dt 160 missing values (<1% of total rows)
- In order_approved_dt missing 4% of months, 11% of weeks, 15% of days

**order_delivered_carrier_dt**

In [ ]:
df_orders.order_delivered_carrier_dt.explore.info()

**Key Observations:**  

- In order_delivered_carrier_dt 1.78k missing values (2% of total rows).
- In order_delivered_carrier_dt missing 2% of weeks, 22% of days.

**order_delivered_customer_dt**

In [ ]:
df_orders.order_delivered_customer_dt.explore.info()

**Key Observations:**  

- In order_delivered_customer_dt 2.96k missing values (3% of total rows).
- In order_delivered_customer_dt missing 3% of weeks, 12% of days.

**order_estimated_delivery_dt**

In [ ]:
df_orders.order_estimated_delivery_dt.explore.info(plot=False)

**Key Observations:**  

- In order_estimated_delivery_dt missing 4% of weeks, 41% of days.

### Adding Temporary Dimensions

To study anomalies across different dimensions, we will add temporary metrics.

We will prefix their names with 'tmp_' to indicate that these are temporary metrics to be removed later.

They are temporary because the data may change after preprocessing.

Therefore, the primary metrics will be created after preprocessing.

Let’s check the initial DataFrame size and save it to ensure no data is lost later.

In [ ]:
print(df_orders.shape[0])
tmp_ids = df_orders.order_id

In [ ]:
tmp_df_reviews = (
    df_reviews.groupby('order_id', as_index=False)
    .agg(tmp_avg_reviews_score = ('review_score', 'mean'))
)
tmp_df_reviews['tmp_avg_reviews_score'] = np.floor(tmp_df_reviews['tmp_avg_reviews_score']).astype(int).astype('category')

tmp_df_payments = (
    df_payments.groupby('order_id', as_index=False)
    .agg(tmp_payment_types = ('payment_type', lambda x: ', '.join(x.unique())))
)
tmp_df_items = (
    df_items.merge(df_products, on='product_id', how='left')
    .assign(product_category_name = lambda x: x['product_category_name'].cat.add_categories(['missed in df_products']))
    .fillna({'product_category_name': 'missed in df_products'})
    .groupby('order_id', as_index=False)
    .agg(tmp_product_categories = ('product_category_name', lambda x: ', '.join(x.unique())))
)

df_orders = (
    df_orders.merge(tmp_df_reviews, on='order_id', how='left')
    .merge(tmp_df_payments, on='order_id', how='left')
    .merge(tmp_df_items, on='order_id', how='left')
    .merge(df_customers[['customer_id', 'customer_state']], on='customer_id', how='left')
    .rename(columns={'customer_state': 'tmp_customer_state'})
)

df_orders['tmp_product_categories'] = df_orders['tmp_product_categories'].fillna('Missing in Items').astype('category')

df_orders['tmp_payment_types'] = df_orders['tmp_payment_types'].fillna('Missing in Pays').astype('category')

df_orders['tmp_order_purchase_month'] = df_orders['order_purchase_dt'].dt.month_name().fillna('Missing purchase dt').astype('category')

df_orders['tmp_order_purchase_weekday'] = df_orders['order_purchase_dt'].dt.day_name().fillna('Missing purchase dt').astype('category')

conditions = [
    df_orders['order_purchase_dt'].isna()                      
    , df_orders['order_purchase_dt'].dt.hour.between(4,11)
    , df_orders['order_purchase_dt'].dt.hour.between(12,16)
    , df_orders['order_purchase_dt'].dt.hour.between(17,22)
    , df_orders['order_purchase_dt'].dt.hour.isin([23, 0, 1, 2, 3])
]
choices = ['Missing purchase dt', 'Morning', 'Afternoon', 'Evening', 'Night']
df_orders['tmp_purchase_time_of_day'] = np.select(conditions, choices, default='Missing purchase dt')
df_orders['tmp_purchase_time_of_day'] = df_orders['tmp_purchase_time_of_day'].astype('category')

conditions = [
    df_orders['order_delivered_customer_dt'].isna() | df_orders['order_estimated_delivery_dt'].isna()
    , df_orders['order_delivered_customer_dt'] > df_orders['order_estimated_delivery_dt']            
    , df_orders['order_delivered_customer_dt'] <= df_orders['order_estimated_delivery_dt']                           
]
choices = ['Missing delivery dt', 'Delayed', 'Not Delayed']
df_orders['tmp_is_delayed'] = np.select(conditions, choices, default='Missing delivery dt')
df_orders['tmp_is_delayed'] = df_orders['tmp_is_delayed'].astype('category')

conditions = [
    df_orders['order_status'].isna(), 
    df_orders['order_status'] == 'Delivered',               
    df_orders['order_status'] != 'Delivered',  
]
choices = ['Missing Status', 'Delivered', 'Not Delivered']
df_orders['tmp_is_delivered'] = np.select(conditions, choices, default='Missing Status')
df_orders['tmp_is_delivered'] = df_orders['tmp_is_delivered'].astype('category')

del tmp_df_reviews, tmp_df_payments, tmp_df_items

Verified that nothing was lost.

In [ ]:
df_orders.shape[0]

In [ ]:
set(df_orders.order_id) == set(tmp_ids)

All good.

### Exploring Missing Values

Let’s examine which columns contain missing values.

In [ ]:
df_orders.explore.anomalies_report(
    anomaly_type='missing'
    , width=600
)

**Key Observations:**
- Missing values in these columns likely belong to orders that did not reach a certain status.

We will analyze missing values in each column separately.

**Missing in order_approved_dt**

In [ ]:
tmp_miss = df_orders[df_orders['order_approved_dt'].isna()]

Let’s examine missing values in payment approval time over time.
Time will be based on order creation time.

In [ ]:
df_orders['order_approved_dt'].explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , anomaly_type='missing'
    , freq='W'
)

**Key Observations:**

- In February 2017 and August 2018, there was a spike in orders missing payment approval timestamps.

Let’s analyze by order status.

In [ ]:
df_orders['order_approved_dt'].explore.anomalies_by_categories(
    anomaly_type='missing'
    , pct_diff_threshold=-100
    , include_columns='order_status'
)

**Key Observations:**

- Missing values in the "canceled" and "created" statuses are logical.
- However, 14 missing values in order_approved_dt for orders with "delivered" status are unusual.

Let’s examine these 14 delivered orders with missing order_approved_dt.

In [ ]:
tmp_miss[lambda x: x.order_status == 'Delivered']

**Key Observations:**  

- All delivered orders with missing order_approved_dt used "boleto" as the payment method. This may be a characteristic of "boleto" usage.
- All these orders were placed in January and February 2017. There may have been a system issue where approval timestamps were not saved.

Let’s examine the 5 created orders that have missing values in the payment approval time.

Let’s look at 5 "created" orders with missing payment approval timestamps.

In [ ]:
tmp_miss[lambda x: x.order_status == 'Created']

**Key Observations:**  

- Orders with "created" status and missing payment approval timestamps were placed long ago and never delivered. The data may not have been processed.

Let’s analyze by average order review score.

In [ ]:
df_orders['order_approved_dt'].explore.anomalies_by_categories(
    anomaly_type='missing'
    , pct_diff_threshold=-100
    , include_columns='tmp_avg_reviews_score'
)

**Key Observations:**
- The difference in proportions is significantly higher for score 1. These orders were likely not delivered.

Let’s examine a word cloud from review messages.

In [ ]:
tmp_miss = tmp_miss.merge(df_reviews, on='order_id', how='left')

In [ ]:
tmp_miss.viz.wordcloud('review_comment_message')

**Key Observations:**  

- Many words relate to delivery.

Let’s analyze the sentiment of the text.

In [ ]:
tmp_miss.analysis.sentiment('review_comment_message')

**Key Observations:**  

- The sentiment is not predominantly negative.

Let’s randomly sample 20 review comments.
We’ll repeat this several times.

In [ ]:
messages = (
    tmp_miss['review_comment_message']
    .dropna()
    .sample(20)
    .tolist()
)
display(messages)

**Key Observations:**

- Based on review messages, many orders were not delivered, but a significant number were delivered.
- Therefore, missing payment approval timestamps cannot be assumed to indicate order cancellation.

Let’s analyze by payment type.

In [ ]:
df_orders['order_approved_dt'].explore.anomalies_by_categories(
    anomaly_type='missing'
    , pct_diff_threshold=-100
    , include_columns='tmp_payment_types'
)

**Key Observations:**

- The proportion of "voucher" payments in missing values has increased significantly. This payment type is notably more frequent in missing values.
- The "voucher" payment type has a stronger correlation with missing payment approval timestamps. This is likely a characteristic of this payment method.

Let’s analyze by month.

In [ ]:
df_orders['order_approved_dt'].explore.anomalies_by_categories(
    anomaly_type='missing'
    , pct_diff_threshold=-100
    , include_columns='tmp_order_purchase_month'
)

**Key Observations:**
- August has a noticeably higher proportion of missing values than other months. This is also visible in the graph above.

---

**Missing Values in order_delivered_carrier_dt**

In [ ]:
tmp_miss = df_orders[df_orders['order_delivered_carrier_dt'].isna()]

Let’s examine the distribution of missing values in the carrier handover time.

In [ ]:
df_orders['order_delivered_carrier_dt'].explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , anomaly_type='missing'
    , freq='W'
)

**Key Observations:**  

- In November 2017, there was a spike in orders missing carrier handover timestamps. This may be related to Black Friday.

Let’s analyze by order status.

In [ ]:
df_orders['order_delivered_carrier_dt'].explore.anomalies_by_categories(
    anomaly_type='missing'
    , pct_diff_threshold=-100
    , include_columns='order_status'
)

**Key Observations:**
- There are 2 delivered orders with missing order_delivered_carrier_dt.
- All orders with "unavailable" status have missing order_delivered_carrier_dt.

Let’s examine these 2 delivered orders.

In [ ]:
tmp_miss[lambda x: x.order_status == 'Delivered'].merge(df_payments, on='order_id', how='left')

**Key Observations:**  

- Both orders with missing order_delivered_carrier_dt were paid via credit card.

Let’s analyze by average review score.

In [ ]:
df_orders['order_delivered_carrier_dt'].explore.anomalies_by_categories(
    anomaly_type='missing'
    , pct_diff_threshold=-100
    , include_columns='tmp_avg_reviews_score'
)

**Key Observations:**

- The difference in proportions is significantly higher for score 1. These orders were likely not delivered.
- Review score 1 has the strongest correlation with missing carrier handover timestamps. This suggests these orders were not delivered, and customers were highly dissatisfied.

Let’s examine a word cloud from review messages.

In [ ]:
tmp_miss = tmp_miss.merge(df_reviews, on='order_id', how='left')

In [ ]:
tmp_miss.viz.wordcloud('review_comment_message')

**Key Observations:**  

- Many words relate to delivery.

Let’s analyze the sentiment of the text.

In [ ]:
tmp_miss.analysis.sentiment('review_comment_message')

**Key Observations:**  

- Negative reviews outnumber positive ones, and the boxplot body lies mostly below 0.

Let’s randomly sample 20 review comments.
We’ll repeat this several times.

In [ ]:
messages = (
    tmp_miss['review_comment_message']
    .dropna()
    .sample(20)
    .tolist()
)
display(messages)

**Key Observations:**

- Based on review messages, many orders were not delivered, but a significant number were delivered.
- Orders with missing carrier handover timestamps were more frequently undelivered compared to those with missing payment approval timestamps.
- Some products may have been out of stock, and sellers did not hand them over to carriers.
- However, since many orders were still delivered, missing values cannot be assumed to indicate order cancellation.

Let’s analyze by customer state.

In [ ]:
df_orders['order_delivered_carrier_dt'].explore.anomalies_by_categories(
    anomaly_type='missing'
    , include_columns='tmp_customer_state'
)

**Key Observations:**
- The difference in proportions is slightly higher in São Paulo compared to other states.

---

**Missing Values in order_delivered_customer_dt**

In [ ]:
tmp_miss = df_orders[df_orders['order_delivered_customer_dt'].isna()]

Let’s examine the distribution of missing values in customer delivery time.

In [ ]:
df_orders['order_delivered_customer_dt'].explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , anomaly_type='missing'
    , freq='W'
)

**Key Observations:**  

- In November 2017, there was a spike in orders missing customer delivery timestamps.

Let’s analyze by order status.

In [ ]:
df_orders['order_delivered_customer_dt'].explore.anomalies_by_categories(
    anomaly_type='missing'
    , pct_diff_threshold=-100
    , include_columns='order_status'
)

**Key Observations:**
- There are 8 orders with "delivered" status but missing delivery timestamps.

Let’s analyze by customer state.

In [ ]:
df_orders['order_delivered_customer_dt'].explore.anomalies_by_categories(
    anomaly_type='missing'
    , include_columns='tmp_customer_state'
)

**Key Observations:**
- The difference in proportions is slightly higher in Rio de Janeiro.

Let’s examine these 8 delivered orders.

In [ ]:
tmp_miss[lambda x: x.order_status == 'Delivered'].merge(df_payments, on='order_id', how='left')

**Key Observations:**  

- 7 out of 8 orders with missing order_delivered_customer_dt were paid via credit card, and 1 was paid via debit card.

Let’s analyze by average review score.

In [ ]:
df_orders['order_delivered_customer_dt'].explore.anomalies_by_categories(
    anomaly_type='missing'
    , pct_diff_threshold=-100
    , include_columns='tmp_avg_reviews_score'
)

**Key Observations:**
- The difference in proportions is significantly higher for score 1. These orders were likely not delivered.

Let’s examine a word cloud from review messages.

In [ ]:
tmp_miss = tmp_miss.merge(df_reviews, on='order_id', how='left')

In [ ]:
tmp_miss.viz.wordcloud('review_comment_message')

**Key Observations:**  

- Many words relate to delivery.

Let’s analyze the sentiment of the text.

In [ ]:
tmp_miss.analysis.sentiment('review_comment_message')

**Key Observations:**  

- Negative reviews outnumber positive ones, and the boxplot body lies mostly below 0.

Let’s randomly sample 20 review comments.  
We’ll repeat this several times.

In [ ]:
messages = (
    tmp_miss['review_comment_message']
    .dropna()
    .sample(20)
    .tolist()
)
display(messages)

**Key Observations:**

- Based on review messages, some orders were not delivered, but this is less frequent than with missing payment approval or carrier handover timestamps.
- Many messages confirm order receipt. Thus, these orders cannot be assumed canceled.

In [ ]:
del tmp_miss

### Anomalies in Order Status

We have many orders with statuses other than "delivered." This is unusual. Let's investigate this.

Let's examine by status.

In [ ]:
df_orders.order_status.value_counts()  

Let’s look at missing values in the timestamps by order status.

In [ ]:
columns = [
    "order_status",
    "order_purchase_dt",
    "order_approved_dt",
    "order_delivered_carrier_dt",
    "order_delivered_customer_dt",
    "order_estimated_delivery_dt",
]
(
    df_orders[columns].pivot_table(
        index='order_status',
        aggfunc=lambda x: x.isna().sum(),
        observed=True,
    )
    .reset_index()
    [columns]
)

Let’s look at the number of orders without the delivered status over time.

In [ ]:
labels = dict(
    order_purchase_dt = 'Date',
    order_id = 'Number of Orders', 
    order_status = 'Order Status', 
)
df_orders[lambda x: x.order_status != 'Delivered'].viz.line(
    x='order_purchase_dt',
    y='order_id',
    color='order_status',
    agg_func='nunique',
    freq='ME',
    labels=labels,
    markers=True,
    title='Number of Orders without Delivered Status by Month and Order Status',   
)

**Key Observations:**  

- In March and April 2018, there was a sharp spike in orders stuck in the "shipped" status.
- In February and August 2018, there were spikes in the "canceled" status.
- In November 2017, there was a spike in the "unavailable" status. This month included Black Friday.

Let's examine each status separately.

**created**

Let’s look at the rows in the dataframe with orders that have the status ‘created’.

In [ ]:
df_orders[lambda x: x.order_status == 'Created']

**Key Observations:**  

- One order has a rating of 5, while four orders have a rating of 1.
- The process stops after purchase, before payment approval.

Let’s look at the review messages.

In [ ]:
messages = (
    df_orders[lambda x: x.order_status == 'Created']
    .merge(df_reviews, on='order_id', how='left')
    ['review_comment_message']
    .tolist()
)
display(messages)

**Key Observations:**  

- Based on review comments, these orders were not delivered.

---

**approved**

Let’s look at the rows.

In [ ]:
df_orders[lambda x: x.order_status == 'Approved']

**Key Observations:**  

- One order received a rating of 1, the other a 4.
- The process stops after payment approval, before carrier handover.

Let’s look at the review messages.

In [ ]:
messages = (
    df_orders[lambda x: x.order_status == 'Approved']
    .merge(df_reviews, on='order_id', how='left')
    ['review_comment_message']
    .tolist()
)
display(messages)

**Key Observations:**  

- No comments were left for these orders.

---

**processing**

Let’s look at orders with the status ‘processing’ by month.

In [ ]:
df_orders['order_status'].explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , custom_mask=df_orders.order_status == 'Processing'
    , freq='ME'
)

Let’s look at the count of each order status.

In [ ]:
tmp_anomal = df_orders[lambda x: x.order_status == 'Processing']

In [ ]:
(
    tmp_anomal[['order_purchase_dt', 'order_approved_dt', 'order_delivered_carrier_dt', 'order_delivered_customer_dt', 'order_estimated_delivery_dt']]
    .count()    
    .to_frame('count')
)

**Key Observations:**  

- The process stops after payment approval, before carrier handover.

Let’s look at it broken down by the average order rating.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
    custom_mask=df_orders.order_status == 'Processing'
    , include_columns='tmp_avg_reviews_score'
)

**Key Observations:**  

- 86% of orders with "processing" status have a rating of 1.
- 6% of orders have a rating of 2.
- Customers are clearly dissatisfied.

Let’s look at it broken down by payment type.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
    custom_mask=df_orders.order_status == 'Processing'
    , include_columns='tmp_payment_types'
)

**Key Observations:**
- The "boleto" payment type has a slightly higher proportion difference.

Let’s randomly sample 20 review comments.  
We’ll repeat this several times.

In [ ]:
tmp_anomal = tmp_anomal.merge(df_reviews, on='order_id', how='left')

In [ ]:
messages = (
    tmp_anomal['review_comment_message']
    .dropna()
    .sample(20)
    .tolist()
)
display(messages)

**Key Observations:**  

- Based on review messages, orders were not delivered.
- Some reviews mention items being out of stock.

Let’s examine a word cloud from review messages.

In [ ]:
tmp_anomal.viz.wordcloud('review_comment_message')

**Key Observations:**  

- Most words relate to delivery.

Let’s analyze the sentiment of the text.

In [ ]:
tmp_anomal.analysis.sentiment('review_comment_message')

**Key Observations:**  

- Negative reviews significantly outnumber positive ones, and the boxplot lies in the negative zone.

---

**invoiced**

Let’s look at orders with the status ‘invoiced’ by month.

In [ ]:
df_orders['order_status'].explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , custom_mask=df_orders.order_status == 'Invoiced'
    , freq='ME'
)

Let’s look at the count of each order status.

In [ ]:
tmp_anomal = df_orders[lambda x: x.order_status == 'Invoiced']

In [ ]:
(
    tmp_anomal[['order_purchase_dt', 'order_approved_dt', 'order_delivered_carrier_dt', 'order_delivered_customer_dt', 'order_estimated_delivery_dt']]
    .count()    
    .to_frame('count')
)

**Key Observations:**  

- The process stops after payment approval, before carrier handover.

Let’s look at it broken down by the average order rating.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
    custom_mask=df_orders.order_status == 'Invoiced'
    , include_columns='tmp_avg_reviews_score'
)

**Key Observations:**  

- 74% of orders with "invoiced" status have a rating of 1.
- 9% of orders have a rating of 2.
- Customers are clearly dissatisfied.

Let’s randomly sample 20 review comments.  
We’ll repeat this several times.

In [ ]:
tmp_anomal = tmp_anomal.merge(df_reviews, on='order_id', how='left')

In [ ]:
messages = (
    tmp_anomal['review_comment_message']
    .dropna()
    .sample(20)
    .tolist()
)
display(messages)

**Key Observations:**  

- Review messages indicate orders were not delivered.
- Some reviews mention items being out of stock.

Let’s examine a word cloud from review messages.

In [ ]:
tmp_anomal.viz.wordcloud('review_comment_message')

**Key Observations:**  

- Many words relate to delivery.

Let’s analyze the sentiment of the text.

In [ ]:
tmp_anomal.analysis.sentiment('review_comment_message')

**Key Observations:**  

- Negative reviews significantly outnumber positive ones, and the boxplot mostly lies below 0.

---

**unavailable**

Let’s look at orders with the status ‘unavailable’ by month.

In [ ]:
df_orders['order_status'].explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , custom_mask=df_orders.order_status == 'Unavailable'
    , freq='ME'
)

Let’s look at the count of each order status.

In [ ]:
tmp_anomal = df_orders[lambda x: x.order_status == 'Unavailable']

In [ ]:
( 
    tmp_anomal[['order_purchase_dt', 'order_approved_dt', 'order_delivered_carrier_dt', 'order_delivered_customer_dt', 'order_estimated_delivery_dt']]
    .count()    
    .to_frame('count') 
) 

**Key Observations:**  

- The process stops after payment approval, before carrier handover.

Let’s look by the customer’s state.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
    custom_mask=df_orders.order_status == 'Unavailable'
    , pct_diff_threshold=1
    , include_columns='tmp_customer_state'
)

**Key Observations:**  

- The proportion of missing values in São Paulo is higher than in the full dataset.

Let’s look  by product category.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
    custom_mask=df_orders.order_status == 'Unavailable'
    , pct_diff_threshold=0
    , include_columns='tmp_product_categories'
)

**Key Observations:**  

- 99% of orders lack a category, meaning they are not in the items table.

Let’s look at it broken down by payment type.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
    custom_mask=df_orders.order_status == 'Unavailable'
    , pct_diff_threshold=0
    , include_columns='tmp_payment_types'
)

**Key Observations:**  

- The "boleto" payment type has a slightly higher proportion difference.

Let’s look at it broken down by the average order rating.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
    custom_mask=df_orders.order_status == 'Unavailable'
    , pct_diff_threshold=0
    , include_columns='tmp_avg_reviews_score'
)

**Key Observations:**  

- The difference in proportions is much higher for a rating of 1.
- 78% of orders with "unavailable" status have a rating of 1.
- 8% of orders have a rating of 2.
- Customers are clearly dissatisfied.

Let’s randomly sample 20 review comments.  
We’ll repeat this several times.

In [ ]:
tmp_anomal = tmp_anomal.merge(df_reviews, on='order_id', how='left')

In [ ]:
messages = (
    tmp_anomal['review_comment_message']
    .dropna()
    .sample(20)
    .tolist()
)
display(messages)

**Key Observations:**  

- Review messages indicate orders were not delivered.
- Some reviews mention items being out of stock.

Let’s examine a word cloud from review messages.

In [ ]:
tmp_anomal.viz.wordcloud('review_comment_message')

**Key Observations:**  

- Many words relate to delivery.

Let’s analyze the sentiment of the text.

In [ ]:
tmp_anomal.analysis.sentiment('review_comment_message')

**Key Observations:**  

- Negative reviews outnumber positive ones, and the boxplot mostly lies below 0.

---

**canceled**

Let’s look at orders with the status ‘canceled’ by month.

In [ ]:
df_orders['order_status'].explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , custom_mask=df_orders.order_status == 'Canceled'
    , freq='ME'
)

Order cancellation can occur at different stages, so there may be missing values at various points. 

Let’s look at the missing values.

In [ ]:
tmp_anomal = df_orders[lambda x: x.order_status == 'Canceled']

In [ ]:
tmp_anomal.explore.detect_anomalies()

**Conversion at different stages**

Let’s look at the count of different order status timestamps. 

Let’s check if there are any missing values between the dates.

In [ ]:
mask = tmp_anomal['order_delivered_carrier_dt'].isna() & tmp_anomal['order_delivered_customer_dt'].notna()
tmp_anomal.loc[mask, 'order_delivered_carrier_dt'] 

In [ ]:
mask = tmp_anomal['order_approved_dt'].isna() & tmp_anomal['order_delivered_carrier_dt'].notna()
tmp_anomal.loc[mask, 'order_approved_dt']

All good.

In [ ]:
tmp_funnel = ( 
    tmp_anomal[['order_purchase_dt', 'order_approved_dt', 'order_delivered_carrier_dt', 'order_delivered_customer_dt']]
    .count()    
    .to_frame('count') 
    .assign(share = lambda x: (x['count']*100 / x['count']['order_purchase_dt']).round(1).astype(str) + '%')
    .reset_index(names='stage')
) 

In [ ]:
px.funnel(
    tmp_funnel, 
    x='count', 
    y='stage', 
    text='share',
    width=600,
    title='Conversion of Different Order Stages with "Canceled" Status'
)

**Key Observations:**  

- The process stops at different stages, most often between payment approval and carrier handover.

Let’s look at the conversion at each stage by month. 

For this, we will count the number of canceled orders with specific timestamps in each period and divide by the number of canceled orders at the time of purchase.

In [ ]:
tmp_res_df = (
    tmp_anomal.resample('ME', on='order_purchase_dt')
    .agg(
        purchase = ('order_id', 'count')
        , approved = ('order_approved_dt', 'count')
        , delivered_carrier = ('order_delivered_carrier_dt', 'count')
        , delivered_customer = ('order_delivered_customer_dt', 'count')
    )
) 
tmp_res_df = tmp_res_df.div(tmp_res_df['purchase'], axis=0)
tmp_res_df = (  
    tmp_res_df.reset_index(names='date')
    .melt(id_vars='date', var_name='date_type', value_name='count')
)

Let’s look at the non-normalized values. That is, divide each value (count with a specific timestamp) by the total value for the period.

In [ ]:
labels = dict(
    date = 'Date',
    date_type = 'Date Type',
    count = 'Conversion'
)
tmp_res_df.viz.line(
    x='date'
    , y='count'
    , color='date_type'
    , labels=labels
    , title='Conversion of Different Order Stages with "Canceled" Status by Month'
)

**Key Observations:**  

- Canceled orders almost never have delivery timestamps, which is logical.
- From December 2017 to March 2018, there was a significant spike in canceled orders that had carrier handover timestamps but no delivery timestamps, indicating delivery issues during this period.
- About 80% of canceled orders have payment approval timestamps, but this proportion increased significantly starting January 2018, approaching 100%.

**Number of Last Stages**

Let’s look at the last stage to which orders with the status ‘canceled’ reach over time. 

For this:
- transform the wide table into a long one, making the name of the time variable a category;
- remove missing values in the time (this will be the variable with the value after melt);
- convert these categories into a categorical type in pandas and specify the order;
- group by order;
- take the first time in each group (all entries in the group will have the same time);
- take the maximum stage (since we specified the order, this will be the last stage of the order).

In [ ]:
tmp_df_orders_canceled = df_orders[lambda x: x.order_status == 'Canceled']
tmp_df_orders_canceled['tmp_date'] = tmp_df_orders_canceled['order_purchase_dt']

In [ ]:
tmp_df_orders_canceled = (
    tmp_df_orders_canceled.rename(
        columns={
            'order_purchase_dt': 'purchase'
            , 'order_approved_dt': 'approved'
            , 'order_delivered_carrier_dt': 'delivered_carrier'
            , 'order_delivered_customer_dt': 'delivered_customer'
        }
    )
    .melt(
        id_vars=['tmp_date', 'order_id']
        , value_vars=['purchase', 'approved', 'delivered_carrier', 'delivered_customer']
        , var_name='date_stage'
    )
    .dropna(subset='value')
    .drop('value', axis=1)
)

In [ ]:
date_stage_order = ['purchase', 'approved', 'delivered_carrier', 'delivered_customer']
tmp_df_orders_canceled['date_stage'] = (
    tmp_df_orders_canceled['date_stage']
    .astype('category')
    .cat.reorder_categories(date_stage_order, ordered=True)
)

In [ ]:
tmp_df_orders_canceled = (
    tmp_df_orders_canceled.groupby('order_id', as_index=False)
    .agg(
        tmp_date = ('tmp_date', 'first')
        , last_date_stage = ('date_stage', 'max')
    )
)

Let’s look at it over time.

In [ ]:
labels = dict(
    date = 'Date',
    order_id = 'Number of Orders',
    last_date_stage = 'Last Stage'
)
tmp_df_orders_canceled.viz.line(
    x='tmp_date'
    , y='order_id'
    , color='last_date_stage'
    , agg_func='nunique'
    , freq='ME'
    , labels=labels
    , markers=True
    , title='Number of Orders by Month and Last Stage'
)

**Key Observations:**  

- In most months, the process stops after payment approval.
- From December 2017 to March 2018, there was a spike in orders that stopped after carrier handover.
- In August 2018, there was a sharp peak in orders that stopped immediately after purchase.

Let’s look by the customer’s state.

In [ ]:
tmp_anomal = df_orders[lambda x: x.order_status == 'Canceled']

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
   custom_mask=df_orders.order_status == 'Canceled'
   , pct_diff_threshold=0
    , include_columns='tmp_customer_state'
)

**Key Observations:**  

- The proportion of missing values in São Paulo is significantly higher than in the full dataset.

Let’s look  by product category.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
   custom_mask=df_orders.order_status == 'Canceled'
   , pct_diff_threshold=1
    , include_columns='tmp_product_categories'
)

**Key Observations:**  

- Missing product categories have a much higher proportion difference, possibly due to items being out of stock.

Let’s look at it broken down by payment type.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
   custom_mask=df_orders.order_status == 'Canceled'
   , pct_diff_threshold=0
    , include_columns='tmp_payment_types'
)

**Key Observations:**  

- The "voucher" payment type has a noticeably higher proportion difference.

Let’s look at it broken down by the average order rating.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
   custom_mask=df_orders.order_status == 'Canceled'
   , pct_diff_threshold=0
    , include_columns='tmp_avg_reviews_score'
)

**Key Observations:**  

- 69% of orders with "canceled" status have a rating of 1.
- 7% of orders have a rating of 2.
- Customers are clearly dissatisfied.

Let’s randomly sample 20 review comments.  
We’ll repeat this several times.

In [ ]:
tmp_anomal = tmp_anomal.merge(df_reviews, on='order_id', how='left')

In [ ]:
messages = (
    tmp_anomal['review_comment_message']
    .dropna()
    .sample(20)
    .tolist()
)
display(messages)

**Key Observations:**  

- Review messages indicate orders were not delivered.
- Some reviews mention items being out of stock.

Let’s examine a word cloud from review messages.

In [ ]:
tmp_anomal.viz.wordcloud('review_comment_message')

**Key Observations:**  

- Many words relate to delivery.

Let’s analyze the sentiment of the text.

In [ ]:
tmp_anomal.analysis.sentiment('review_comment_message')

**Key Observations:**  

- Negative messages significantly outnumber positive ones, and the boxplot lies below 0.

---

**shipped**

Let’s look at the number of orders with the status ‘delivered’ by month.

In [ ]:
df_orders['order_status'].explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , custom_mask=df_orders.order_status == 'Shipped'
    , freq='ME'
)

Let’s look at the count of each order status.

In [ ]:
tmp_anomal = df_orders[lambda x: x.order_status == 'Shipped']

In [ ]:
(
    tmp_anomal[['order_purchase_dt', 'order_approved_dt', 'order_delivered_carrier_dt', 'order_delivered_customer_dt', 'order_estimated_delivery_dt']]
    .count()    
    .to_frame('count')
)

**Key Observations:**  

- The process stops after carrier handover, before customer delivery.

Let’s look by the customer’s state.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
    custom_mask=df_orders.order_status == 'Shipped'
    , pct_diff_threshold=1
    , include_columns='tmp_customer_state'
)

**Key Observations:**  

- The proportion of missing values in Rio de Janeiro is significantly higher than in the full dataset.

Let’s look at it broken down by the average order rating.

In [ ]:
df_orders['order_status'].explore.anomalies_by_categories(
    custom_mask=df_orders.order_status == 'Shipped'
    , pct_diff_threshold=1
    , include_columns='tmp_avg_reviews_score'
)

**Key Observations:**  

- 62% of orders with "shipped" status have a rating of 1.
- 8% of orders have a rating of 2.
- Customers are clearly dissatisfied.

Let’s randomly sample 20 review comments.  
We’ll repeat this several times.

In [ ]:
tmp_anomal = tmp_anomal.merge(df_reviews, on='order_id', how='left')

In [ ]:
messages = (
    tmp_anomal['review_comment_message']
    .dropna()
    .sample(20)
    .tolist()
)
display(messages)

**Key Observations:**  

- Review messages indicate most orders were not delivered.

Let’s examine a word cloud from review messages.

In [ ]:
tmp_anomal.viz.wordcloud('review_comment_message')

**Key Observations:**  

- Many words relate to delivery.

Let’s analyze the sentiment of the text.

In [ ]:
tmp_anomal.analysis.sentiment('review_comment_message')

**Key Observations:**  

- Negative messages outnumber positive ones, and the boxplot mostly lies below 0.

### Status and Delivery Mismatches

**Delivery status missing but delivery timestamp present**

Let's check if there are orders without "delivered" status that still have a delivery timestamp.

In [ ]:
df_orders[lambda x: (x.order_status != 'Delivered') & ~x.order_delivered_customer_dt.isna()]

**Key Observations:**  

- There are orders without "delivered" status that have delivery timestamps. Most likely these orders were canceled after delivery.

Let's examine their reviews

In [ ]:
messages = (
    df_orders[lambda x: (x.order_status != 'Delivered') & ~x.order_delivered_customer_dt.isna()]
    .merge(df_reviews, on='order_id', how='left')
    ['review_comment_message']
    .dropna()
    .tolist()
)
display(messages)

**Key Observations:**

- Review messages indicate these orders were not delivered.

---

**Status is "delivered" but delivery timestamp is missing**

Let's check if there are orders with "delivered" status but missing delivery timestamps.

In [ ]:
df_orders[lambda x: x.order_status.isin(['Delivered']) & x.order_delivered_customer_dt.isna()]

**Key Observations:**  

- The dataset contains 8 orders with "delivered" status but missing delivery timestamps.

Let's examine their reviews

In [ ]:
messages = (
    df_orders[lambda x: x.order_status.isin(['Delivered']) & x.order_delivered_customer_dt.isna()]
    .merge(df_reviews, on='order_id', how='left')
    ['review_comment_message']
    .dropna()
    .tolist()
)
display(messages)

**Key Observations:**  

- Review messages suggest the products were actually delivered.

---

**Order canceled or unavailable but has delivery timestamp**

Let's check if there are orders with "canceled" or "unavailable" status that still have delivery timestamps.

In [ ]:
df_orders[lambda x: x.order_status.isin(['Canceled', 'Unavailable']) & ~x.order_delivered_customer_dt.isna()]

**Key Observations:**  

- The dataset contains 6 orders with "canceled" status that have customer delivery timestamps

Let's examine their reviews

In [ ]:
messages = (
    df_orders[lambda x: x.order_status.isin(['Canceled', 'Unavailable']) & ~x.order_delivered_customer_dt.isna()]
    .merge(df_reviews, on='order_id', how='left')
    ['review_comment_message']
    .dropna()
    .tolist()
)
display(messages)

**Key Observations:**  

- Review messages indicate some items were delivered while others were not.

### Date Inconsistencies

**order_purchase_dt**

Let's check if there are timestamps earlier than purchase dates.

In [ ]:
for col_dt in ['order_approved_dt', 'order_delivered_carrier_dt', 'order_delivered_customer_dt']:
    rows_cnt = df_orders[~(df_orders['order_purchase_dt'].isna() | df_orders[col_dt].isna()
                          | (df_orders['order_purchase_dt'] <= df_orders[col_dt]))].shape[0]
    if rows_cnt:
        print(f'{col_dt} < order_purchase_dt, rows count: {rows_cnt}')

**Key Observations:**  

- There are 166 orders where carrier handover time is earlier than purchase time. This is unusual.

Let's examine the dataframe

In [ ]:
tmp_mask = ~(df_orders['order_purchase_dt'].isna() | df_orders['order_delivered_carrier_dt'].isna()
                          | (df_orders['order_purchase_dt'] <= df_orders['order_delivered_carrier_dt']))
tmp_df_orders = df_orders[tmp_mask]
print(f'rows: {tmp_df_orders.shape[0]}')
display(tmp_df_orders.head(5))

Let's analyze by day

In [ ]:
tmp_df_orders.explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , custom_mask=tmp_mask
    , freq='D'
)

**Key Observations:**
- These anomalies only occurred between 25 April and 24 August 2018.

Let's analyze by order status

In [ ]:
df_orders.explore.anomalies_by_categories(
    custom_mask=tmp_mask
    , pct_diff_threshold=-100
    , include_columns='order_status'
)

Let’s look at it broken down by payment type.

In [ ]:
df_orders.explore.anomalies_by_categories(
    custom_mask=tmp_mask
    , pct_diff_threshold=-100
    , include_columns='tmp_payment_types'
)

**Key Observations:**
- Over 90% of anomalous orders were paid by credit card.

Let's analyze by time of day

In [ ]:
df_orders.explore.anomalies_by_categories(
    custom_mask=tmp_mask
    , pct_diff_threshold=-100
    , include_columns='tmp_purchase_time_of_day'
)

**Key Observations:**
- Most anomalies occurred in the afternoon.

Let's examine their reviews

In [ ]:
messages = (
    tmp_df_orders.merge(df_reviews, on='order_id', how='left')
    ['review_comment_message']
    .dropna()
    .sample(20)
    .tolist()
)
display(messages)

**Key Observations:**  

- Nothing unusual found.

---

**order_approved_dt**

Let's check if there are timestamps that should occur after approval but appear earlier.

In [ ]:
for col_dt in ['order_delivered_carrier_dt', 'order_delivered_customer_dt']:
    rows_cnt = df_orders[~(df_orders['order_approved_dt'].isna() | df_orders[col_dt].isna()
                          | (df_orders['order_approved_dt'] <= df_orders[col_dt]))].shape[0]
    if rows_cnt:
        print(f'{col_dt} < order_approved_dt, rows count: {rows_cnt}')

**Key Observations:**  

- There are 1,359 orders where carrier handover time is earlier than payment approval time.
- There are 61 orders where delivery time is earlier than payment approval time.

Let’s examine each one separately.

**order_delivered_carrier_dt < order_approved_dt**

Let's examine the dataframe

In [ ]:
tmp_mask = ~(df_orders['order_approved_dt'].isna() | df_orders['order_delivered_carrier_dt'].isna()
                          | (df_orders['order_approved_dt'] <= df_orders['order_delivered_carrier_dt']))
tmp_df_orders = df_orders[tmp_mask]
print(f'rows: {tmp_df_orders.shape[0]}')
display(tmp_df_orders.head(5))

Let's examine by days

In [ ]:
tmp_df_orders.explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , custom_mask=tmp_mask
    , freq='D'
)

**Key Observations:**
- Days with most anomalies:
    - 19-23 April 2018
    - 3-4 July 2018
- Possible system issues caused delayed payment approvals.

Let's analyze by order status

In [ ]:
df_orders.explore.anomalies_by_categories(
    custom_mask=tmp_mask
    , pct_diff_threshold=-100
    , include_columns='order_status'
)

**Key Observations:**
- Nearly all orders were eventually delivered.

Let's analyze by time of day

In [ ]:
df_orders.explore.anomalies_by_categories(
    custom_mask=tmp_mask
    , pct_diff_threshold=-100
    , include_columns='tmp_purchase_time_of_day'
)

**Key Observations:**
- More anomalies occurred in the afternoon.

**order_delivered_customer_dt < order_approved_dt**

In [ ]:
tmp_mask = ~(df_orders['order_approved_dt'].isna() | df_orders['order_delivered_customer_dt'].isna()
                          | (df_orders['order_approved_dt'] <= df_orders['order_delivered_customer_dt']))
tmp_df_orders = df_orders[tmp_mask]

Let's examine by days

In [ ]:
tmp_df_orders.explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , custom_mask=tmp_mask
    , freq='D'
)

**Key Observations:**
- Anomalies occurred sporadically on specific dates.

Let’s look at it broken down by payment type.

In [ ]:
df_orders.explore.anomalies_by_categories(
    custom_mask=tmp_mask
    , pct_diff_threshold=-100
    , include_columns='tmp_payment_types'
)

**Key Observations:**
- "Boleto" payments had significantly more anomalies.

Let's analyze by customer state

In [ ]:
df_orders.explore.anomalies_by_categories(
    custom_mask=tmp_mask
    , pct_diff_threshold=-100
    , include_columns='tmp_customer_state'
)

**Key Observations:**
- Most anomalies occurred in São Paulo.

**order_delivered_carrier_dt**

Let's check if there are timestamps that should occur after carrier handover but appear earlier.

In [ ]:
tmp_mask = ~(df_orders['order_delivered_carrier_dt'].isna() | df_orders['order_delivered_customer_dt'].isna()
                        | (df_orders['order_delivered_carrier_dt'] <= df_orders['order_delivered_customer_dt']))
rows_cnt = df_orders[tmp_mask].shape[0]
if rows_cnt:
    print(f'order_delivered_customer_dt < order_delivered_carrier_dt, rows count: {rows_cnt}')

**Key Observations:**  

- There are 23 orders where delivery time is earlier than carrier handover time.

**review_creation_dt < order_purchase_dt**

We have order creation time and review creation time. Let's check if any reviews were created before their corresponding orders.

In [ ]:
temp_df = df_orders.merge(df_reviews, on='order_id', how='left')
temp_df = temp_df[lambda x: x.order_purchase_dt.dt.date > x.review_creation_dt]
temp_df.shape[0]

The dataset contains 65 orders where reviews were created before the orders themselves.

Let's examine them

In [ ]:
temp_df.head()

Let’s look at how many orders do not have an approval payment date.

In [ ]:
temp_df.order_approved_dt.isna().sum()

Let’s look at how many orders do not have a delivery date.

In [ ]:
temp_df.order_approved_dt.isna().sum()

Let’s look at how many of them were canceled.

In [ ]:
temp_df.order_status.value_counts() 

**Key Observations:**  

- The dataset contains 65 orders where reviews were created before the orders themselves. 58 orders were canceled. 6 were delivered. 1 was in delivery process.

Let's examine the 6 delivered orders

In [ ]:
temp_df[temp_df.order_status=='Delivered']

We previously determined that one order can have multiple reviews and one review can cover multiple orders.

Let's check for duplicates in these orders and reviews.

In [ ]:
temp_unque_orders = temp_df.order_id.unique()
temp_unque_reviews = temp_df.review_id.unique()

In [ ]:
df_reviews[df_reviews.review_id.isin(temp_unque_reviews)].merge(df_orders, on='order_id', how='left').sort_values('review_id').head()

Even accounting for duplicates, both orders show review creation dates preceding order dates.

Let's check if any review responses were created before the reviews themselves:

In [ ]:
df_reviews[lambda x: x.review_creation_dt >=x.review_answer_dt]

No such cases found.

##  Table df_payments

Let’s look at the information about the dataframe.

In [ ]:
df_payments.explore.info()

### Initial Column Analysis

We will examine each column individually.

**order_id**

In [ ]:
df_payments['order_id'].explore.info(plot=False)

**Key Observations:**  

- All is well.


**payment_sequential**

In [ ]:
df_payments['payment_sequential'].explore.info(plot=False)

**Key Observations:**  

- The maximum number of payment methods for a single order is 29.

**payment_type**

In [ ]:
df_payments['payment_type'].explore.info(plot=True)

**Key Observations:**  

- 74% of payments were made using credit cards.
- The payment_type field contains undefined payment types (<1%).

**payment_installments**

In [ ]:
df_payments['payment_installments'].explore.info()

**Key Observations:**  

- The maximum number of installments for a product payment is 24.
- The median number of payment installments is 1.
- 75% of orders have installment plans with 4 or fewer payments.
- There are 2 orders with a value of 0 in payment_installments.

**payment_value**

In [ ]:
df_payments['payment_value'].explore.info()

**Key Observations:**  

- There are 9 zero-value payments in payment_value.
- The maximum payment is 13.66k. The median payment is 100.
- The 13.66k payment is clearly an outlier.

### Exploring Outliers

In [ ]:
df_payments.explore.anomalies_report(
    anomaly_type='outlier'
)

Let's examine payments exceeding 5,000.

In [ ]:
df_payments[df_payments.payment_value > 5_000]

Let's check for outliers in total order amounts per user.

In [ ]:
(
    df_customers.merge(df_orders, on='customer_id', how='left')
    .merge(df_payments, on='order_id', how='left')
    .groupby('customer_unique_id')['payment_value']
    .sum()
    .sort_values(ascending=False)
    .to_frame()
    .head(10)
)


**Key Observations:**  

- One user made orders totaling 13,664. This clearly stands out from the rest.
- There are also several users who made purchases totaling 6,000 or more.

Let's identify outliers using quantiles.

We'll consider values outside the 5th and 95th percentiles as outliers.

In [ ]:
df_payments.explore.detect_anomalies(
    anomaly_type='outlier'
    , method='quantile'
    , threshold=0.05
)

**Key Observations:**  

- 10% of payment values are outliers. This exceeds the typical norm (5%) but isn't critical.
- For payment installments, outliers account for less than 1%, which is normal.

**payment_value**

Let's examine the distribution of payment value outliers over time.

In [ ]:
tmp_outl = df_payments.merge(df_orders, on='order_id', how='left')

In [ ]:
tmp_outl['payment_value'].explore.anomalies_over_time(
    time_column='order_purchase_dt'
    , anomaly_type='outlier'
    , freq='D'
)

**Key Observations:**  

- Many payment outliers occurred between November 20-26, 2017, likely related to Black Friday.

In [ ]:
del tmp_outl

### Exploring Other Anomalies

Let's explore zero values.

In [ ]:
df_payments.explore.anomalies_report(
    anomaly_type='zero'
    , sample_size=20
)

**Key Observations:**  

- Orders with zero payment amounts have either "voucher" or "not_defined" as their payment type.

Let's examine zeros in each column separately.

**Zeros in payment_installments**

In [ ]:
df_payments[df_payments.payment_installments == 0]

Since payment_sequential shows 2, there should have been another payment. Let's examine these orders.

In [ ]:
df_payments[df_payments.order_id == '744bade1fcf9ff3f31d860ace076d422']

In [ ]:
df_payments[df_payments.order_id == '1a57108394169c0b47d8f876acc9ba2d']

Let's check these orders in df_items.

In [ ]:
df_items[df_items.order_id == '744bade1fcf9ff3f31d860ace076d422']

In [ ]:
df_items[df_items.order_id == '1a57108394169c0b47d8f876acc9ba2d']

As we can see, the order wasn't fully recorded in df_payments. The first payment is missing.

---

**Zeros in payment_value**

In [ ]:
df_payments[df_payments.payment_value == 0]

Let's look at other payments for order fa65dad1b0e818e3ccc5cb0e39231352.

In [ ]:
df_payments[df_payments.order_id == '8bcbe01d44d147f901cd3192671144db']

**Key Observations:**  

- One payment was processed as zero, and it was the last payment.- 
- Moreover, all zero payments have either "voucher" or "not_defined" as their type.- 
- There might be some specific payment logic here.- 
- It's better not to modify these zeros.

##  Table df_items

Let’s look at the information about the dataframe.

In [ ]:
df_items.explore.info()

### Initial Column Analysis

We will examine each column individually.

**order_id**

In [ ]:
df_items['order_id'].explore.info(plot=False)

**order_item_id**

In [ ]:
df_items['order_item_id'].explore.info()

**Key Observations:**  

- The maximum quantity of items in a single order is 21.


**product_id**

In [ ]:
df_items['product_id'].explore.info(plot=False)

**seller_id**

In [ ]:
df_items['seller_id'].explore.info(plot=False)

**shipping_limit_dt**

In [ ]:
df_items['shipping_limit_dt'].explore.info()

**Key Observations:**  

- In shipping_limit_dt: 20% missing years, 41% missing months, 47% missing weeks, 57% missing days.
- The maximum date in shipping_limit_dt is 2020-04-09.


**price**

In [ ]:
df_items['price'].explore.info()

**Key Observations:**  

- Most products are priced between 39.9 and 134.9.
- The median product price is 74.99.

**freight_value**

In [ ]:
df_items['freight_value'].explore.info()

**Key Observations:**  

- There are zero values in freight_value.

### Exploring Outliers


In [ ]:
df_items.explore.anomalies_report(
    anomaly_type='outlier'
    , exclude_columns='seller_id'
)

**Key Observations:**  

- About 10% outliers exist in product prices and shipping costs. This exceeds the typical norm (usually 5%) but isn't critical.

### Product Sales Inconsistencies

Checking if any products in the items table have multiple sellers:

In [ ]:
df_items.groupby('product_id')['seller_id'].nunique().sort_values(ascending=False).head(10).to_frame('sellers_cnt')

**Key Observations:**  

- Some product IDs were sold by different sellers.

In [ ]:
df_items.groupby('product_id')['seller_id'].nunique().value_counts().to_frame('products_cnt')

**Key Observations:**  

- Over 1,000 products have more than 2 sellers.

Examining product d285360f29ac7fd97640bf0baef03de0

In [ ]:
df_products[lambda x: x.product_id == 'd285360f29ac7fd97640bf0baef03de0']

In [ ]:
tmp_df_res = (df_items[lambda x: x.product_id == 'd285360f29ac7fd97640bf0baef03de0']
        [['shipping_limit_dt', 'price', 'freight_value', 'seller_id']]
        .merge(df_sellers, on='seller_id', how='left')
)
tmp_df_res.seller_id.unique()

In [ ]:
tmp_df_res.seller_state.unique()

In [ ]:
tmp_df_res.seller_city.unique()

**Key Observations:**  

- Sellers are located in different cities.
- This might not be an anomaly - different sellers could legitimately sell identical products with matching IDs.

Checking if any products were sold across different seller states

In [ ]:
(df_items.merge(df_sellers, on='seller_id', how='left')
 .groupby('product_id')['seller_state'].nunique().sort_values(ascending=False).head(10).to_frame('states_cnt')
)

**Key Observations:**  

- Some products were sold by sellers in different states.

Verifying city consistency for customer_id in the customers table (as this is our join key)

In [ ]:
(df_customers.groupby('customer_id')[['customer_state', 'customer_city']].nunique() > 1).sum()

All is well..

### Date Inconsistencies

**shipping_limit_dt**

Analyzing anomalous shipping_limit_dt values

In [ ]:
df_items[df_items.shipping_limit_dt > '2018-12-31'].merge(df_orders, on='order_id', how='left')

**Key Observations:**  

- Found 4 orders with abnormally large shipping_limit_dt values, despite having normal estimated delivery times.

### Exploring Other Anomalies

Examining zero values

In [ ]:
tmp_zeros = df_items.explore.detect_anomalies(
    anomaly_type='zero'
    , return_mode='by_column'
)['freight_value']

**Key Observations:**  

- Zero freight values may indicate free shipping.

Examing rows

In [ ]:
tmp_zeros.sample(5)

Reviewing zero-value over time

In [ ]:
df_items.freight_value.explore.anomalies_over_time(
    time_column='shipping_limit_dt'
    , anomaly_type='zero'
    , freq='W'
)

**Key Observations:**  

- Most zero shipping costs occurred between April-July 2018.

##  Table df_customers

Let’s look at the information about the dataframe.

In [ ]:
df_customers.explore.info()

### Initial Column Analysis

We will examine each column individually.

**customer_id**

In [ ]:
df_customers['customer_id'].explore.info(plot=False)

**customer_unique_id**

In [ ]:
df_customers['customer_unique_id'].explore.info(plot=False)

**Key Observations:**  

- customer_unique_id has 3% duplicates - acceptable as this field doesn't require uniqueness in this table.


**customer_zip_code_prefix**

In [ ]:
df_customers['customer_zip_code_prefix'].explore.info(plot=False)

**customer_city**

In [ ]:
df_customers['customer_city'].explore.info()

**Key Observations:**  

- Most customers are from São Paulo city (16%).


**customer_state**

In [ ]:
df_customers['customer_state'].explore.info()

**Key Observations:**  

- Most customers are from SP state (42%).


##  Table df_reviews

Let’s look at the information about the dataframe.

In [ ]:
df_reviews.explore.info()

### Initial Column Analysis

We will examine each column individually.

**review_id**

In [ ]:
df_reviews['review_id'].explore.info(plot=False)

**Key Observations:**  

- review_id contains 827 duplicates.


**order_id**

In [ ]:
df_reviews['order_id'].explore.info(plot=False)

**Key Observations:**  

- order_reviews table has 559 duplicate order_ids.


**review_score**

In [ ]:
df_reviews['review_score'].explore.info(column_type='categorical')

**Key Observations:**  

- Over half of reviews (57%) give maximum 5-star ratings.

**review_comment_title**

In [ ]:
df_reviews['review_comment_title'].explore.info(column_type='text')

**Key Observations:**  

- 88% of review titles are missing.
- Most common review title (8%) is 'recomendo'.

**review_comment_message**

In [ ]:
df_reviews['review_comment_message'].explore.info() 

**Key Observations:**  

- 58% of orders lack review messages.
- Only 36% of review comments are unique.
- Most frequent comment (1%) contains "muito bom".

**review_creation_dt**

In [ ]:
df_reviews['review_creation_dt'].explore.info()

**Key Observations:**  

- review_creation_dt has 9% missing days.

**review_answer_dt**

In [ ]:
df_reviews['review_answer_dt'].explore.info()

**Key Observations:**  

- review_answer_dt has 5% missing days.

### Exploring Missing Values

Checking columns with missing values:

In [ ]:
df_reviews.explore.anomalies_report(
    anomaly_type='missing'
    , pct_diff_threshold=10
    , show_by_categories=False
    , show_sample=False
    , width=600
)

**Key Observations:**  

- Missing review titles/messages aren't anomalies - they were simply not provided.

### Exploring Duplicates

Examining duplicates in order_id and review_id:

In [ ]:
df_reviews[['order_id', 'review_id']].duplicated().sum()

No instances where both order_id and review_id are duplicated simultaneously.

Theoretical possibility: one order could have multiple reviews, but multiple orders sharing one review is unusual.

Analyzing order_id and review_id duplicates separately

**review_id**

In [ ]:
tmp_dupl = df_reviews[df_reviews.review_id.duplicated()]

Reviewing review_id duplicate distribution over time

In [ ]:
df_reviews.review_id.explore.anomalies_over_time(
    time_column='review_creation_dt'
    , anomaly_type='duplicate'
    , freq='W'
)

**Key Observations:**  

- March 2018 saw a significant spike in duplicate review_ids (one review applied to multiple orders).

Checking for duplicates with different customers

In [ ]:
tmp_dupl = (
    tmp_dupl.merge(df_orders, on='order_id', how='left')
    .merge(df_customers, on='customer_id', how='left')
)
tmp_dupl.groupby('review_id')['customer_unique_id'].nunique().value_counts()

**Key Observations:**  

- No duplicates with different customers.

Examining product quantities in these orders

In [ ]:
(
    tmp_dupl.merge(df_items, on='order_id', how='left')
    .groupby('order_id')['product_id']
    .nunique()
    .value_counts()
)

**Key Observations:**  

- Most orders contain one product.
- Some orders show no products (due to missing records in the items table).

In [ ]:
tmp_dupl = (tmp_dupl.merge(df_payments, on='order_id', how='left')
            .merge(df_items, on='order_id', how='left')
)

In [ ]:
tmp_dupl = tmp_dupl[['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message'
                   , 'review_creation_dt', 'order_delivered_customer_dt', 'order_status', 'payment_type'
                   , 'payment_value', 'product_id', 'price', 'freight_value']].sort_values('review_id').drop_duplicates()
tmp_dupl.head()

Comparing duplicate values across columns (replacing missing values with __na__):

In [ ]:
(tmp_dupl.fillna({'review_comment_message': '__na__'})
 .groupby('review_id')
 [['review_comment_message', 'review_score', 'order_status', 'payment_type', 'payment_value', 'product_id', 'price']]
 .nunique()
 .apply(pd.Series.value_counts)
)

**Key Observations:**  

- Identical reviews were left for different orders with matching ratings and descriptions, but varying products/prices.- 
- This is unusual - could indicate bulk reviews for multiple orders or data collection errors.

---

**order_id**

Analyzing order_id duplicate distribution over time

In [ ]:
tmp_dupl = df_reviews[df_reviews.order_id.duplicated()]

In [ ]:
df_reviews.order_id.explore.anomalies_over_time(
    time_column='review_creation_dt'
    , anomaly_type='duplicate'
    , freq='W'
)

**Key Observations:**  

- March 2018 showed a spike in order_id duplicates (multiple reviews for single orders).

In [ ]:
tmp_dupl = (tmp_dupl.merge(df_orders, on='order_id', how='left')
            .merge(df_customers, on='customer_id', how='left')
)

In [ ]:
tmp_dupl = tmp_dupl[['order_id', 'review_id', 'review_score', 'review_comment_title', 'review_comment_message'
                   , 'review_creation_dt', 'order_delivered_customer_dt', 'order_status']].sort_values('order_id').drop_duplicates()
tmp_dupl.head(10)

**Key Observations:**  

- Customers sometimes left multiple reviews per order (e.g., one pre-delivery and one post-delivery) - not necessarily anomalous.

Let's examine how many duplicates share identical values across different columns.

For description fields, we'll replace missing values with __na__.

In [ ]:
(tmp_dupl.fillna({'review_comment_message': '__na__'})
 .groupby('order_id')
 [['review_comment_message', 'review_score', 'order_status']]
 .nunique()
 .apply(pd.Series.value_counts)
)

Secondary Review Rating Comparison

We'll analyze whether follow-up reviews for the same order had higher or lower ratings:

- Compare average ratings with initial ratings
- If initial rating was lower, subsequent ratings were either equal or higher

In [ ]:
(tmp_dupl.sort_values(['order_id', 'review_creation_dt'])
 .groupby('order_id')
 .agg(
     first_review_score = ('review_score', 'first')
     , mean_review_score = ('review_score', 'mean')
 )
 .assign(
     is_first_less_mean = lambda x: x.first_review_score < x.mean_review_score
 )
 ['is_first_less_mean']
 .value_counts()
)

**Key Observations:**  

- Subsequent reviews for the same order typically received lower ratings than the initial review.

In [ ]:
del tmp_dupl

##  Table df_products

Let’s look at the information about the dataframe.

In [ ]:
df_products.explore.info()

### Initial Column Analysis

We will examine each column individually.

**product_id**

In [ ]:
df_products['product_id'].explore.info(plot=False)

**product_category_name**

In [ ]:
df_products['product_category_name'].explore.info()

**Key Observations:**  

- product_category_name contains 2% missing values
- Dataset contains 73 unique product categories

**product_name_lenght**

In [ ]:
df_products['product_name_lenght'].explore.info()

**Key Observations:**  

- product_name_lenght has 2% missing values
- Maximum product name length: 76 characters

**product_description_lenght**

In [ ]:
df_products['product_description_lenght'].explore.info()

**Key Observations:**  

- There are 2% missing values in product_description_lenght.
- The maximum length of the product description is 3.99k characters.
- The minimum length of the product description is 4 characters.

**product_photos_qty**

In [ ]:
df_products['product_photos_qty'].explore.info(column_type='categorical')

**Key Observations:**  

- In product_photos_qty, 2% of values are missing.
- The maximum number of photos for a single product is 20.
- 50% of products have 1 photo.

**product_weight_g**

In [ ]:
df_products['product_weight_g'].explore.info()

**Key Observations:**  

- In product_weight_g, there are 2 missing values.
- In product_weight_g, there are 4 zero values.
- The maximum product weight is 40.42k grams.
- The product weight of 40.42k grams is clearly an outlier.

**product_length_cm**

In [ ]:
df_products['product_length_cm'].explore.info()

**Key Observations:**  

- In product_length_cm, there are 2 missing values.
- The maximum product length is 105 cm. The minimum is 7 cm. The median is 25 cm.

**product_height_cm**

In [ ]:
df_products['product_height_cm'].explore.info()

**Key Observations:**  

- In product_height_cm, there are 2 missing values.
- The maximum product height is 105 cm. The minimum is 2 cm. The median is 13 cm.

**product_width_cm**

In [ ]:
df_products['product_width_cm'].explore.info()

**Key Observations:**  

- In product_width_cm, there are 2 missing values.
- The maximum product width is 118 cm. The minimum is 6 cm. The median is 20 cm.

### Exploring Missing Values

Let's see which columns have missing values.

In [ ]:
df_products.explore.anomalies_report(
    anomaly_type='missing'
    , pct_diff_threshold=10
)

Let's hypothesize that missing values in the following columns are in the same rows:

- product category name
- product name length
- product description length
- number of product photos

In [ ]:
df_products.explore.detect_simultaneous_anomalies(['product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty'])

**Key Observations:**  

- Missing values in product category name, product name length, product description length, and number of product photos are in the same rows.

Let's hypothesize that missing values in the following columns are in the same rows:

- product length
- product width
- product height
- product weight

In [ ]:
df_products.explore.detect_simultaneous_anomalies(['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'])

**Key Observations:**  

- Missing values in product length, width, height, and weight are located in the same rows.

### Exploring Outliers


In [ ]:
df_products.explore.anomalies_report(
    anomaly_type='outlier'
)

**Key Observations:**  

- The proportion of outliers in the number of photos, length, and width of products is within normal limits.
- The proportion of outliers in product weight and height exceeds the norm (usually 5%), but is not critical.

### Exploring Other Anomalies

Examining zero values

In [ ]:
df_products.explore.anomalies_report(
    anomaly_type='zero'
)

**Key Observations:**  

- All 4 products with zero weight belong to the category cama_mesa_banho (home textiles).

##  Table df_categories

Let’s look at the information about the dataframe.

In [ ]:
df_categories.explore.info()

### Initial Column Analysis

We will examine each column individually.

**product_category_name**

In [ ]:
df_categories['product_category_name'].explore.info(plot=False)

**Key Observations:**  

- The product_category_name table has 71 unique product categories, while the products table has 73 categories.

**product_category_name_english**

In [ ]:
df_categories['product_category_name_english'].explore.info(plot=False)

##  Table df_sellers

Let’s look at the information about the dataframe.

In [ ]:
df_sellers.explore.info()

### Initial Column Analysis

We will examine each column individually.

**seller_id**

In [ ]:
df_sellers['seller_id'].explore.info(plot=False)

**seller_zip_code_prefix**

In [ ]:
df_sellers['seller_zip_code_prefix'].explore.info(plot=False)

**seller_city**

In [ ]:
df_sellers['seller_city'].explore.info()

**Key Observations:**  

- The most sellers are from the city of sao paulo (22%).


**seller_state**

In [ ]:
df_sellers['seller_state'].explore.info()

**Key Observations:**  

- The most sellers are from the state of sp (60%).


##  Table df_geolocations

Let’s look at the information about the dataframe.

In [ ]:
df_geolocations.explore.info()

**Key Observations:**  

- The df_geolocations table has 28% fully duplicated rows.

### Initial Column Analysis

We will examine each column individually.

**geolocation_zip_code_prefix**

In [ ]:
df_geolocations['geolocation_zip_code_prefix'].explore.info(plot=False)

**geolocation_lat**

In [ ]:
df_geolocations['geolocation_lat'].explore.info(plot=False)

**geolocation_lng**

In [ ]:
df_geolocations['geolocation_lng'].explore.info(plot=False)

**geolocation_city**

In [ ]:
df_geolocations['geolocation_city'].explore.info()

**Key Observations:**  

- In geolocation_city, the most entries are for the city of sao paulo (16%).


**geolocation_state**

In [ ]:
df_geolocations['geolocation_state'].explore.info()

**Key Observations:**  

- In geolocation_state, the most entries are for the state of SP (40%).


### Exploring Duplicates

We have complete row duplicates in the geolocation table. Let's examine them.

Let's check if we have duplicates in the geolocation table in the geolocation_zip_code_prefix field, excluding common duplicates.

In [ ]:
tmp_geo = df_geolocations.drop_duplicates()

In [ ]:
tmp_geo.explore.detect_anomalies('duplicate', columns=['geolocation_zip_code_prefix'])

**Key Observations:**  

- In the df_geolocations table, there are 97% duplicates in the geolocation_zip_code_prefix column.

Let's see why there are duplicates.

In [ ]:
tmp_geo.groupby('geolocation_zip_code_prefix').nunique().head(10)

This makes sense, as geolocation_zip_code_prefix can have many different unique coordinates.

But we need to take this into account when joining tables, since we only have zip_code_prefix in the customer and seller tables.

When joining, we may get many duplicates.

We can average the coordinates, but we can't do the same with cities and states.

Let's check if we have multiple states for a single prefix.

In [ ]:
tmp_geo.groupby('geolocation_zip_code_prefix').geolocation_state.nunique().sort_values(ascending=False).head()

Let's see the maximum number of cities with the same prefix.

In [ ]:
tmp_geo.groupby('geolocation_zip_code_prefix').geolocation_city.nunique().sort_values(ascending=False).head()

**Key Observations:**  

- In the df_geolocations table, there are prefixes with 2 unique states.
- In the df_geolocations table, there are prefixes with 4 unique cities.

There's nothing we can do about this. We'll need to account for this when analyzing geolocation coordinates.

Since we have states in the customer and seller tables, we can avoid using city and state from the geolocation table.

And we can average the coordinates.

### Exploring Outliers


Let's see how many sales we have outside South America.

In [ ]:
tmp_geo = df_geolocations.copy()
tmp_geo['in_south_america'] = (
    (tmp_geo.geolocation_lat >= -53.90) &  # Southern border
    (tmp_geo.geolocation_lat <= 12.45) &   # Northern border
    (tmp_geo.geolocation_lng >= -81.32) &  # Western border
    (tmp_geo.geolocation_lng <= -34.79)    # Eastern border
)

In [ ]:
temp_df = (df_orders[['order_id', 'customer_id']].merge(df_customers, on='customer_id', how='left')
              .merge(tmp_geo.drop_duplicates(subset=["geolocation_zip_code_prefix"]), left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
              .dropna()
              [['in_south_america', 'customer_city', 'customer_state', 'geolocation_lat', 'geolocation_lng']]
)
temp_df.in_south_america.value_counts()

**Key Observations:**  

- There are 6 sales outside South America in the dataset.

Let's look at these orders.

In [ ]:
temp_df[temp_df.in_south_america == False]

These coordinates are outside South America.
Either it's an error, or the order was placed outside South America.

- (42.18, -8.72) is off the coast of Spain/Portugal
- (20.09, -30.54) is in the central Atlantic Ocean
- (13.00, -23.58) is in the eastern Atlantic Ocean near Cape Verde
- (-11.31, -34.73) is in the South Atlantic
- (20.09, -30.54) is a repeating point in the central Atlantic Ocean

In [ ]:
del temp_df, tmp_geo

## Exploring Cross-Table Anomalies

### Temporal Boundary Checks

Examining time interval boundaries across different tables.

In [ ]:
for key, df in dfs:
    datetime_cols = [col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])]
    for col in datetime_cols:
        min_date = df[col].min()
        max_date = df[col].max()    
        print(f"[{min_date.date()} - {max_date.date()}] DataFrame '{key}', Column '{col}':")

**Key Observations:**  

- The latest date in order_approved_dt is earlier than in order_purchase_dt.- 
- Anomalously large maximum date in shipping_limit_dt compared to other temporal variables.

### Payment-Order Amount Mismatches

Checking for orders where payment total differs from order value.

In [ ]:
temp_df = (
    df_items[['order_id', 'price', 'freight_value']]
    .groupby('order_id')
    .sum()
    .assign(total_price=lambda x: x['price'] + x['freight_value'])
    .drop(columns=['price', 'freight_value'])
    .reset_index()
    .merge(df_payments, on='order_id', how='inner')
    .merge(df_orders, on='order_id', how='inner')
    .dropna(subset=['payment_value', 'total_price'])
)
temp_df['payment_matches_total'] = temp_df['payment_value'].round(2) == temp_df['total_price'].round(2)
temp_df['payment_matches_total'].value_counts()

**Key Observations:**  

- Dataset contains 7,877 orders with payment-amount discrepancies.

In [ ]:
tmp_mask = ~temp_df.payment_matches_total

Let's analyze by payment type.

In [ ]:
temp_df.explore.anomalies_by_categories(
    custom_mask=tmp_mask
    , pct_diff_threshold=-100
    , include_columns='tmp_payment_types'
)

**Key Observations:**  

- Payment type analysis shows most mismatches involve voucher payments (likely systemic issue).

## Exploring Relationships Between Tables

Reviewing inter-table connections for future joins and key consistency.

**df_orders and df_payments**

In [ ]:
fron.analyze_join_keys(df_orders, df_payments, "order_id", short_result=False)

**Key Observations:**  

- Orders table contains 1 order_id missing from payments table.

Let's look at what this order is.

In [ ]:
temp_df = df_orders.merge(df_payments, on='order_id', how='left')
temp_df[temp_df.payment_value.isna()]

**df_orders and df_items**

In [ ]:
fron.analyze_join_keys(df_orders, df_items, "order_id", short_result=False)

**Key Observations:**  

- Orders table contains 775 order_ids missing from items table.

We have a payments table. Let's check if orders missing from items exist in payments.

In [ ]:
missing_orders = (df_orders.merge(df_items, on='order_id', how='left')
                  [lambda x: x['order_item_id'].isna()].order_id.unique()
)
len(missing_orders)

In [ ]:
df_payments[df_payments.order_id.isin(missing_orders)].order_id.nunique()

All these orders are present in the payments table.

Let's check how many of these orders are canceled.

In [ ]:
df_orders[df_orders['order_id'].isin(missing_orders)].order_status.value_counts()

**Key Observations:**  

- These orders are either canceled, unavailable, or just created.

Let's examine orders with "shipped" status.

In [ ]:
df_orders[df_orders['order_id'].isin(missing_orders) & (df_orders.order_status == 'Shipped')]

**df_orders and df_customers**

In [ ]:
fron.analyze_join_keys(df_orders, df_customers, "customer_id", short_result=False)

**Key Observations:**  

- All is well..

**df_orders and df_reviews**

In [ ]:
fron.analyze_join_keys(df_orders, df_reviews, "order_id", short_result=False)

**Key Observations:**  

- All is well..

**df_items and df_products**

In [ ]:
fron.analyze_join_keys(df_items, df_products, "product_id", short_result=False)

**Key Observations:**  

- All is well..

**df_items and df_sellers**

In [ ]:
fron.analyze_join_keys(df_items, df_sellers, "seller_id", short_result=False)

**Key Observations:**  

- All is well..

**df_customers and df_geolocations**

In [ ]:
fron.analyze_join_keys(df_customers, df_geolocations, left_on = 'customer_zip_code_prefix', right_on = "geolocation_zip_code_prefix", short_result=False)

**Key Observations:**  

- In df_customers table, there are 157 zip_code_prefixes not present in df_geolocations.- 
- In df_geolocations table, there are 4178 zip_code_prefixes not present in df_customers.

**df_sellers and df_geolocations**

In [ ]:
fron.analyze_join_keys(df_sellers, df_geolocations, left_on = 'seller_zip_code_prefix', right_on = "geolocation_zip_code_prefix", short_result=False)

**Key Observations:**  

- In df_sellers table, there are 7 zip_code_prefixes not present in df_geolocations.- 
- In df_geolocations table, there are 16776 zip_code_prefixes not present in df_sellers.

Delete temporary fields.

In [ ]:
df_orders = df_orders[[col for col in df_orders.columns if not col.startswith('tmp_')]]

Clear memory of temporary variables.

In [ ]:
for var_name in list(globals().keys()):
    if var_name.startswith('tmp_'):
        del globals()[var_name]

In [ ]:
%run ../../_post_run.ipynb